In [1]:
import os
import pandas as pd
from tqdm import tqdm
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms.functional as TF
from torchvision.models import densenet121, vgg16, resnet50, inception_v3
import glob
from torch.autograd import Variable
from efficientnet_pytorch import EfficientNet
import copy
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils, datasets
from skimage import io, transform
from PIL import Image
from sklearn.utils import class_weight
from torch.autograd import Variable
from torch.optim.lr_scheduler import ReduceLROnPlateau

In [2]:
BASE_DIR = '/srv/app/data'

DATA_DIR = BASE_DIR + '/data'

MODEL_DIR = BASE_DIR + '/models/'

TEST_DIR = DATA_DIR + '/numpy_array/stage_2_test_images_299_roi_interpolated/'
IMAGE_FORMAT = 'npy'

BATCH_SIZE = 150

LABEL_COLUMN = ['any', 'epidural', 'intraparenchymal', 'intraventricular', 'subarachnoid', 'subdural']

targets = ['ID', 'epidural', 'intraparenchymal', 'intraventricular', 'subarachnoid', 'subdural', 'any']

files_list = os.listdir(TEST_DIR)

files_ids = [x.split('.')[0] for x in files_list]

CUDA_DEVICES = [1,2,3]

In [3]:
is_cuda=False
if torch.cuda.is_available():
    is_cuda = True
print(is_cuda)    

# Detect if we have a GPU available
cuda_list = ','.join(str(c) for c in CUDA_DEVICES)
device = torch.device("cuda:{}".format(cuda_list) if torch.cuda.is_available() else "cpu")

True


# Load Test Data

In [4]:
len(files_list)

121232

In [5]:
class CustomPredictDataset(Dataset):

    def __init__(self, X, img_folder, img_ext='png', transform=None, index=None):
        """
        Args:
            X (dataframe): Dataframe with images ID.
            img_folder (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.X = X
        self.img_folder = img_folder
        self.img_ext = img_ext
        self.transform = transform
        self.index = index

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        img_name = os.path.join(self.img_folder, self.X.iloc[idx, 0] + '.' + self.img_ext)
        #image = np.load(img_name).astype('uint8')
        image = np.load(img_name)
        
        if self.index:
            image = image[:,:,[int(self.index)]]
            image = np.repeat(image, 3, axis=2)
        if self.transform:
        
            image = self.transform(TF.to_pil_image(image))

        return image

In [6]:
X = pd.DataFrame(files_ids, columns =['ID']) 

# Load model

In [7]:
def predictProbas(model, transform, layer=None):
#     torch.cuda.empty_cache()
    dataset = CustomPredictDataset(
                            X=X, 
                            img_folder=TEST_DIR, 
                            img_ext=IMAGE_FORMAT,
                            transform=transform,
                            index=layer
    )
    loader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)
    outputs = torch.zeros(1, 6).to(device)
    with torch.no_grad():
        for inputs in tqdm(loader):
            inputs = inputs.to(device)
            output = model(inputs)
            if type(output) == tuple:
                output = output[0]
            output = torch.sigmoid(output)
            outputs = torch.cat((outputs, output))
    outputs = outputs[1:,:]
    return outputs

In [8]:
def testTimeAugmentationPredict(model, transform_list, layer=None):
    model = torch.load(MODEL_DIR+model)
    model.eval()
    sumTensor = torch.zeros(len(X),6).to(device)
    for transform in transform_list:
        print('Transform {}'.format(str(transform)))
        predict = predictProbas(model, transform, layer)
        sumTensor = sumTensor + predict
    finalPredict = sumTensor / len(transform_list)
    return finalPredict

In [9]:
def ensembleModelsTestTimeAugmentation(models_list, transform_list, layer=None):
    sumTensor = torch.zeros(len(X),6).to(device)
    for model in models_list:
        print('Model: {}'.format(model))
        predict = testTimeAugmentationPredict(model, transform_list, layer)
        sumTensor = sumTensor + predict
    finalPredict = sumTensor / len(models_list)
    return finalPredict

In [10]:
test_transf = transforms.Compose([
    transforms.ToTensor()
])

test_transfA1 = transforms.Compose([
    transforms.RandomRotation((0,360)),
    transforms.ToTensor()
])

test_transfA2 = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ToTensor()
])

test_transfA3 = transforms.Compose([
    transforms.RandomVerticalFlip(p=0.5),
    transforms.ToTensor()
])

transforms_list = [test_transf, test_transfA1, test_transfA2, test_transfA3]

In [11]:
models = ['FineTuningDensenet121MultiTaskV2_SGDMomentumV7_WeightedMultiLabelLogLoss_imgsize299_loss0.06919282247931359.pt',
         'FineTuningResNet50AttentionMultiTaskV2_SGDMomentumV7_WeightedMultiLabelLogLoss_imgsize299_loss0.07118233637800009.pt']

ensemble = ensembleModelsTestTimeAugmentation(models, transforms_list)

Model: FineTuningDensenet121MultiTaskV2_SGDMomentumV7_WeightedMultiLabelLogLoss_imgsize299_loss0.06919282247931359.pt


  0%|          | 0/809 [00:00<?, ?it/s]

Transform Compose(
    ToTensor()
)


  0%|          | 0/809 [00:00<?, ?it/s]

Transform Compose(
    RandomRotation(degrees=(0, 360), resample=False, expand=False)
    ToTensor()
)


...

  0%|          | 0/809 [00:00<?, ?it/s]

Transform Compose(
    RandomHorizontalFlip(p=0.5)
    ToTensor()
)


  0%|          | 0/809 [00:00<?, ?it/s]

Transform Compose(
    RandomVerticalFlip(p=0.5)
    ToTensor()
)


100%|██████████| 809/809 [04:13<00:00,  3.20it/s]


# Save to DataFrame

In [12]:
Y = pd.DataFrame(ensemble.tolist(), columns = LABEL_COLUMN)
Y = Y.reset_index(drop=True)

In [13]:
len(Y)

121232

In [14]:
pred = X.merge(Y, left_index = True, right_index = True)
pred = pred[targets]
pred = pd.melt(pred, id_vars=['ID'], value_vars=targets[1:])
pred['ID'] = pred['ID']+'_'+pred['variable']
pred = pred.drop('variable', axis =1)
pred.columns = ['ID', 'Label']

In [15]:
len(pred) #471270

727392

# Save to CSV

In [16]:
pred.to_csv(DATA_DIR + '/predicts/stage_2_pred01.csv', index=False)